In [4]:
import numpy as np
import pandas as pd
import json
import networkx as nx
import itertools


In [5]:
# fonction pour générer les résultats du secret santa
def secret_santa(liste_participants, config):
    '''
    la fonction prend en arguments une liste de participants, et une configuration qui définit:
    - les relations offrant > recevant prédéfinies
    - les relations offrant > recevant à exclure  
    elle renvoie une liste de couples (offrant, recevant)
    '''
    resultats = []
    offrants_restant = liste_participants.copy()
    recevants_restant = liste_participants.copy()
    exclusions = config['exclusions'].copy()
    obligations = config['obligations'].copy()

    # on traite d'abord les obligations
    print('traitement des obligations')
    for offrant, recevant in obligations:
        print('offrant: {}, recevant: {}'.format(offrant, recevant))
        resultats.append((offrant, recevant))
        offrants_restant.pop(offrants_restant.index(offrant))
        recevants_restant.pop(recevants_restant.index(recevant))
        
    # on parcours au hasard la liste des participants
    print('parcours de la liste')
    for i_offrant_a_traiter in np.random.permutation(len(offrants_restant)):
        offrant_a_traiter = offrants_restant[i_offrant_a_traiter]
        print('offrant: {}'.format(offrant_a_traiter))
        # on tire au hasard les recevants restants
        for i_recevant_possible in np.random.permutation(len(recevants_restant)):
            recevant_a_traiter = recevants_restant[i_recevant_possible]
            print('recevant: {}'.format(recevant_a_traiter))
            # vérification des exclusions
            if (offrant_a_traiter, recevant_a_traiter) not in exclusions and offrant_a_traiter != recevant_a_traiter:
                resultats.append((offrant_a_traiter, recevant_a_traiter))
                recevants_restant.pop(i_recevant_possible)
                break
            else:
                continue
    return(resultats)

In [6]:
liste_participants = ['Papa', 'Maman', 'Didier', 'Anne', 'Cécile', 'Mamy', 'Bruno']
config = {
    'exclusions':[
        ('Papa', 'Maman'),
        ('Maman', 'Papa'),
        ('Cecile', 'Bruno'),
        ('Bruno', 'Cecile'),
        ('Anne', 'Didier'),
        ('Didier', 'Anne')],
    'obligations':[
        ('Bruno', 'Papa'),
        ('Didier', 'Maman')]
}
secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
offrant: Bruno, recevant: Papa
offrant: Didier, recevant: Maman
parcours de la liste
offrant: Papa
recevant: Cécile
offrant: Maman
recevant: Didier
offrant: Anne
recevant: Bruno
offrant: Mamy
recevant: Mamy
recevant: Anne
offrant: Cécile
recevant: Mamy


,offrant,recevant
0,Bruno,Papa
1,Didier,Maman
2,Papa,Cécile
3,Maman,Didier
4,Anne,Bruno
5,Mamy,Anne
6,Cécile,Mamy


In [7]:
liste_participants = ['TERRIER Viktor', 'FINET Sebastien', 'GIROD Marie', 'COGEN Florent', 'DUMAS Jonathan', 'LITTLE Emily', 'DUSSARTRE Virginie', 'HAMDANE Ibtissam', 
                      'ARULPRAGASAM Steven', 'MOTHE Axel', 'BUSTARRET Quentin', 'LEMETAYER Bruno']
config = {
    'exclusions':[],
    'obligations':[]}
secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
parcours de la liste
offrant: ARULPRAGASAM Steven
recevant: FINET Sebastien
offrant: BUSTARRET Quentin
recevant: COGEN Florent
offrant: DUSSARTRE Virginie
recevant: BUSTARRET Quentin
offrant: GIROD Marie
recevant: DUSSARTRE Virginie
offrant: DUMAS Jonathan
recevant: DUMAS Jonathan
recevant: TERRIER Viktor
offrant: FINET Sebastien
recevant: HAMDANE Ibtissam
offrant: HAMDANE Ibtissam
recevant: LEMETAYER Bruno
offrant: LITTLE Emily
recevant: MOTHE Axel
offrant: TERRIER Viktor
recevant: ARULPRAGASAM Steven
offrant: COGEN Florent
recevant: DUMAS Jonathan
offrant: LEMETAYER Bruno
recevant: LITTLE Emily
offrant: MOTHE Axel
recevant: GIROD Marie


,offrant,recevant
0,ARULPRAGASAM Steven,FINET Sebastien
1,BUSTARRET Quentin,COGEN Florent
2,DUSSARTRE Virginie,BUSTARRET Quentin
3,GIROD Marie,DUSSARTRE Virginie
4,DUMAS Jonathan,TERRIER Viktor
5,FINET Sebastien,HAMDANE Ibtissam
6,HAMDANE Ibtissam,LEMETAYER Bruno
7,LITTLE Emily,MOTHE Axel
8,TERRIER Viktor,ARULPRAGASAM Steven
9,COGEN Florent,DUMAS Jonathan


In [8]:
with open('data/config_famille.json') as f:
    config = json.load(f)
liste_participants = pd.read_csv('data/participants_famille.csv', header=None, names=['participants'])['participants'].to_list()

secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
offrant: Bruno, recevant: Papa
offrant: Didier, recevant: Maman
parcours de la liste
offrant: Papa
recevant: Cécile
offrant: Anne
recevant: Didier
offrant: Maman
recevant: Anne
offrant: Cécile
recevant: Mamy
offrant: Mamy
recevant: Bruno


,offrant,recevant
0,Bruno,Papa
1,Didier,Maman
2,Papa,Cécile
3,Anne,Didier
4,Maman,Anne
5,Cécile,Mamy
6,Mamy,Bruno


In [9]:
with open('config_exemple.json') as f:
    config = json.load(f)
liste_participants = pd.read_csv('participants_exemple.csv', header=None, names=['participants'])['participants'].to_list()

secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
offrant: Alpha, recevant: Beta
parcours de la liste
offrant: Gamma
recevant: Omega
offrant: Omega
recevant: Delta
offrant: Delta
recevant: Gamma
offrant: Beta
recevant: Alpha


,offrant,recevant
0,Alpha,Beta
1,Gamma,Omega
2,Omega,Delta
3,Delta,Gamma
4,Beta,Alpha


# Avec des graphes

In [43]:
def secret_santa_graph(liste_participants, config):
    '''
    la fonction prend en arguments une liste de participants, et une configuration qui définit:
    - les relations offrant > recevant prédéfinies
    - les relations offrant > recevant à exclure  
    elle renvoie une liste de couples (offrant, recevant)
    '''
    # création du graphe avec les participants
    exclusions = config['exclusions'].copy()
    edge_list = pd.DataFrame(
        [[item[0], item[1], 0] for item in itertools.product(liste_participants, liste_participants) if item[0] != item[1] and [item[0], item[1]] not in exclusions], 
        columns = ['source', 'target', 'retenu'])
    G = nx.from_pandas_edgelist(
        edge_list, 
        edge_attr = 'retenu',
        create_using = nx.DiGraph())
    
    # on traite d'abord les obligations
    obligations = config['obligations'].copy()
    offrants_obliges = []
    recevants_obliges = []
    for offrant, recevant in obligations:
        nx.set_edge_attributes(G, {(offrant, recevant): {'retenu': 1}})
        offrants_obliges.append(offrant)
        recevants_obliges.append(recevant)
        # on enlève toutes les arrêtes qui ne sont pas celle de l'obligation
        # G.remove_edges_from([edge for edge in G.edges if ((edge[0] == offrant and edge[1] != recevant) or (edge[0] != offrant and edge[1] == recevant))])
    # on parcours au hasard la liste des participants
    cadeaux_a_faire = len(liste_participants) - len(obligations)
    parcours_offrant = []
    parcours_recevant = []
    while len(parcours_offrant) < cadeaux_a_faire and len(parcours_recevant) < cadeaux_a_faire:
        offrants_possibles = [offrant for offrant in liste_participants if (offrant not in parcours_offrant and offrant not in offrants_obliges)]
        offrant_a_traiter = offrants_possibles[np.random.randint(len(offrants_possibles))]
        print('offrant: {}'.format(offrant_a_traiter))
        recevants_possibles = [recevant for recevant in list(G[offrant_a_traiter]) if (recevant not in parcours_recevant and recevant not in recevants_obliges)]
        if len(recevants_possibles) > 0:
            # on tire au hasard les recevants restants parmi les target 
            recevant_a_traiter = recevants_possibles[np.random.randint(len(recevants_possibles))]
            print('nouveau cadeau: {}'.format(recevant_a_traiter))
            nx.set_edge_attributes(G, {(offrant_a_traiter, recevant_a_traiter): {'retenu': 1}})
            parcours_offrant.append(offrant_a_traiter)
            parcours_recevant.append(recevant_a_traiter)
        else:
            print('retour arrière')
            nx.set_edge_attributes(G, {(parcours_offrant.pop(), parcours_recevant.pop()): {'retenu': 0}})
            # on enlève toutes les arrêtes qui ne sont pas celle retenue
            # G.remove_edges_from([edge for edge in G.edges if ((edge[0] == offrant_a_traiter and edge[1] != recevant_a_traiter) or (edge[0] != offrant_a_traiter and edge[1] == recevant_a_traiter))])
    return(G, [(offrant, recevant) for offrant, recevant, edge in G.edges(data=True) if edge['retenu'] == 1])


In [46]:
G, secret_santa_results = secret_santa_graph(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

offrant: Gamma
nouveau cadeau: Omega
offrant: Delta
nouveau cadeau: Gamma
offrant: Beta
nouveau cadeau: Delta
offrant: Omega
nouveau cadeau: Alpha


,offrant,recevant
0,Alpha,Beta
1,Beta,Delta
2,Gamma,Omega
3,Omega,Alpha
4,Delta,Gamma


In [110]:
exclusions = config['exclusions'].copy()
edge_list = pd.DataFrame(
        [[item[0], item[1], 0] for item in itertools.product(liste_participants, liste_participants) if item[0] != item[1] and [item[0], item[1]] not in exclusions], 
        columns = ['source', 'target', 'retenu'])
G = nx.from_pandas_edgelist(
    edge_list, 
    edge_attr = ['retenu'],
    create_using = nx.DiGraph())